In [69]:
import torch
import math

pixels1 = torch.tensor([
    [870.0, 847.0],
    [2135.0, 512.0],
    [1664.0, 1015.0],
])

pixels2 = torch.tensor([
    [997.0, 878.0],
    [1959.0, 624.0],
    [1601.0, 1007.0],
])

offset = torch.tensor([
    [440.0, 442.0]
    ])

pixels1 = pixels1 - offset
pixels2 = pixels2 - offset

pixels1,pixels2

def fovyToScale(degree):
    fovy = degree*math.pi/180.0
    aspect = 16.0/9.0
    yScale = 1.0/math.tan(fovy*0.5)
    xScale = yScale / aspect
    return torch.tensor([xScale, yScale]) 


def PixelToProjection(xy):
    wh = torch.tensor([1920.0,1080.0])
    p = xy / wh * torch.tensor([2.0, -2.0]) +  torch.tensor([-1.0, 1.0]) 
    return p

def ProjectionToPixel(xy):
    wh = torch.tensor([1920.0,1080.0])
    p = (xy - torch.tensor([-1.0, 1.0])) / torch.tensor([2.0, -2.0]) * wh
    return p

p1 = PixelToProjection(pixels1)
proj1 = fovyToScale(35.0)
v = p1 / proj1

proj2 = torch.ones(2, requires_grad=True)
#proj2 = fovyToScale(45.0)
#yhat = ProjectionToPixel(v * proj1)
#print(pixels1,yhat)

optimizer = torch.optim.Adam([proj2], lr = 1e-2)

epochs = 1000
for epoch in range(epochs):
    p2 = v * proj2
    yhat = ProjectionToPixel(p2)
    loss = torch.mean(torch.nn.functional.mse_loss(yhat, pixels2))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch*10 % epochs ==0:
        print(loss)

proj2

tensor(13941.4912, grad_fn=<MeanBackward0>)
tensor(1270.7742, grad_fn=<MeanBackward0>)
tensor(82.1013, grad_fn=<MeanBackward0>)
tensor(2.0918, grad_fn=<MeanBackward0>)
tensor(0.2380, grad_fn=<MeanBackward0>)
tensor(0.2241, grad_fn=<MeanBackward0>)
tensor(0.2241, grad_fn=<MeanBackward0>)
tensor(0.2241, grad_fn=<MeanBackward0>)
tensor(0.2241, grad_fn=<MeanBackward0>)
tensor(0.2241, grad_fn=<MeanBackward0>)


tensor([1.3568, 2.4178], requires_grad=True)

In [70]:
#xx = fovyToScale(45.0)
math.atan(1.0/proj2[1])*2/math.pi*180

44.939309318876504